In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [2]:
# 1. 데이터 로드
df = pd.read_excel('../data/weekday_traffic.xlsx')
df.head()

,일자,요일,지점명,지점번호,방향,구분,0시,1시,2시,3시,...,17시,18시,19시,20시,21시,22시,23시,혼잡,평균교통량,새로운 혼잡
0,20240801,목,마포대교,C-07,유입,마포대교남단->마포대교북단,1121.100127,739.396817,603.646236,429.588089,...,3982.454958,3465.886595,2569.941716,2727.605192,2393.113295,2159.360088,1520.158999,True,2564.041667,False
1,20240802,금,마포대교,C-07,유입,마포대교남단->마포대교북단,1064.668712,758.031761,487.068186,397.855528,...,3890.244416,3425.769366,2954.749926,2613.879546,2301.021973,2244.874364,1496.910955,True,2516.208333,False
2,20240805,월,마포대교,C-07,유입,마포대교남단->마포대교북단,616.797360,464.364338,337.000433,282.774209,...,3865.243889,3743.189576,2772.238162,2506.984733,2199.044578,2080.754404,1571.072355,False,2355.000000,False
3,20240806,화,마포대교,C-07,유입,마포대교남단->마포대교북단,1063.667580,687.378212,416.206495,339.725827,...,3839.879003,4042.135076,2663.051395,2346.262683,2647.551173,1916.299569,1523.996405,False,2488.666667,False
4,20240807,수,마포대교,C-07,유입,마포대교남단->마포대교북단,1042.007691,754.815160,531.468487,363.953378,...,3840.413752,3650.694194,2892.151038,2472.565715,2729.293114,2335.362316,1478.987119,True,2549.875000,False


In [3]:
# 2. 독립 변수(X)와 종속 변수(y) 설정
X = df.loc[:, '0시':'23시']  # 시간대별 교통량 데이터를 독립 변수로 사용
y = df['혼잡']  # 혼잡 여부를 종속 변수로 설정

In [4]:
# 3. 학습 데이터와 테스트 데이터 분리
# train_test_split()을 사용해 데이터를 훈련 세트(70%)와 테스트 세트(30%)로 분리합니다.
# stratify 옵션은 종속 변수의 클래스 비율을 유지하도록 합니다. (참고: https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y)

In [5]:
# 4. 데이터 스케일링 (표준화)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [6]:
# 5. KNN 분류 모델 생성 및 학습
knn = KNeighborsClassifier(n_neighbors=5)  # K 값을 5로 설정
knn.fit(X_train_scaled, y_train)

KNeighborsClassifier()

In [7]:
# 6. 테스트 데이터에 대한 예측
y_pred = knn.predict(X_test_scaled)

In [8]:
# 7. 모델 성능 평가
# 혼동 행렬 출력
# 참고: https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html
conf_matrix = confusion_matrix(y_test, y_pred)
print("\n혼동 행렬(Confusion Matrix):")
print(conf_matrix)
# 혼동 행렬(Confusion Matrix):
# - TP (True Positive): 실제로 True인 데이터 중에서 모델이 True라고 예측한 것
# - TN (True Negative): 실제로 False인 데이터 중에서 모델이 False라고 예측한 것
# - FP (False Positive): 실제로 False인 데이터 중에서 모델이 True라고 잘못 예측한 것 (False Positive)
# - FN (False Negative): 실제로 True인 데이터 중에서 모델이 False라고 잘못 예측한 것 (False Negative)
# - 행: 실제 값 (True Labels)
# - 열: 예측 값 (Predicted Labels)
# [[0, 1] 위치는 실제로 비혼잡(0)인데 혼잡(1)으로 잘못 예측된 경우를 의미
#  [1, 0]] 위치는 실제로 혼잡(1)인데 비혼잡(0)으로 잘못 예측된 경우를 의미


혼동 행렬(Confusion Matrix):
[[6 2]
 [0 5]]


In [9]:
# 분류 성능 평가 지표 출력 (정확도, 정밀도, 재현율, F1 점수 등)
# 참고: https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html
prediction_report = classification_report(y_test, y_pred)
print("\n분류 성능 평가(Classification Report):")
print(prediction_report)
# 분류 성능 평가(Classification Report):
# - Precision (정밀도): 모델이 True라고 예측한 것 중 실제로 True인 데이터의 비율 (TP / (TP + FP))
#   => False Positive를 줄이는 데 중점을 둔 지표
# - Recall (재현율): 실제로 True인 것 중에서 모델이 True라고 올바르게 예측한 비율 (TP / (TP + FN))
#   => False Negative를 줄이는 데 중점을 둔 지표
# - F1 Score: 정밀도와 재현율의 조화 평균 (2 * (Precision * Recall) / (Precision + Recall))
#   => Precision과 Recall 간의 균형을 중요시할 때 사용
# - Support: 각 클래스에 속하는 실제 데이터의 수


분류 성능 평가(Classification Report):
              precision    recall  f1-score   support

       False       1.00      0.75      0.86         8
        True       0.71      1.00      0.83         5

    accuracy                           0.85        13
   macro avg       0.86      0.88      0.85        13
weighted avg       0.89      0.85      0.85        13



In [10]:
# 8. 정확도 출력
# 참고: https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html
accuracy = accuracy_score(y_test, y_pred)
print(f"\n모델의 정확도(Accuracy): {accuracy:.2f}")
# 정확도(Accuracy):
# - 전체 데이터 중에서 모델이 올바르게 예측한 비율 ((TP + TN) / (전체 데이터 수))
#   => 정확도는 모델이 전반적으로 얼마나 잘 예측했는지를 나타내는 지표


모델의 정확도(Accuracy): 0.85
